In [1]:
%alias package {__import__('sys').executable} -m pip install --quiet --upgrade %l

In [2]:
session_storage = 'downloads'

import os
os.makedirs(session_storage, exist_ok=True)

In [3]:
%package fsspec

import fsspec

fs_dataset = fsspec.filesystem(
    'simplecache', 
    target_protocol='https', 
    cache_storage=session_storage
)

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [4]:
%package numpy pandas 

import numpy as np
import pandas as pd
'''
%package swifter
import swifter
swifter.set_defaults(
    npartitions=None,
    dask_thres=1,
    scheduler='processes',
    progress_bar=True,
    progress_bar_desc=None,
    allow_dask_on_strings=True,
    force_parallel=False
)
'''

%package scikit-learn
import sklearn as skl
import sklearn.base
import sklearn.compose
#import sklearn.ensemble
import sklearn.feature_extraction
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection
import sklearn.pipeline
#import sklearn.svm

#%package scikit-learn-intelex
#import sklearnex
#sklearnex.patch_sklearn()

#%package xgboost
#import xgboost

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
%package tensorflow

import tensorflow as tf
from tensorflow import keras as keras

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


2022-12-01 12:02:50.676657: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-01 12:02:51.580240: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64:/opt/conda/lib/:/opt/conda/lib/
2022-12-01 12:02:51.580342: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib6

In [6]:
%package spacy spacy-transformers

import spacy
import spacy_transformers
spacy.prefer_gpu()

spacy.cli.download('en_core_web_lg', False, False, '--quiet')

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [7]:
%package scikeras

import scikeras as skeras
import scikeras.wrappers

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


## Dataset

In [61]:
# TODO
df_scripts = pd.read_pickle('./dataset.pkl', compression='gzip')

In [76]:
df_scripts

,script_name,script_text,script_paragraphs
0,American Milkshake Script,\r\n\r\n\r\n\r\n A...,"[, AMERICAN MILKS..."
1,"Man Who Knew Too Much, The Script",THE MAN WHO KNEW TOO MUCH\r...,"[ THE MAN WHO KNEW TOO MUCH,..."
2,"Crow Salvation, The Script","\r\n\r\n ""THE CROW: SALVATION"" -- by Chip...","[, ""THE CROW: SALVATION"" -- by Chip Joha..."
3,Die Hard 2 Script,\r\n\r\n \r\n ...,"[, \n DIE..."
4,Game 6 Script,\r\n \r\n \r\n ...,"[ , \n ..."
...,...,...,...
1203,Ninja Assassin Script,\r\n\r\n\r\n\r\n NINJA...,"[, NINJA ASSASSIN, ..."
1204,"Cooler, The Script",\r\n\r\n\r\n ...,"[, THE..."
1205,Hitchcock Script,\r\n\r\n\r\n\r\n ...,"[, HITCHCOCK..."
1206,Breaking Away Script,"\r\n\r\n \r\n ""BAMB...","[, \n ""BAMBINO"", ..."


In [63]:
df_scripts = df_scripts.astype({
    'script_name': 'category'
})

In [64]:
df_scripts = df_scripts.explode('script_text', ignore_index=True)

In [77]:
df_scripts.dropna(inplace=True)
df_scripts.reset_index(drop=True, inplace=True)

In [78]:
df_scripts

,script_name,script_text,script_paragraphs
0,American Milkshake Script,\r\n\r\n\r\n\r\n A...,"[, AMERICAN MILKS..."
1,"Man Who Knew Too Much, The Script",THE MAN WHO KNEW TOO MUCH\r...,"[ THE MAN WHO KNEW TOO MUCH,..."
2,"Crow Salvation, The Script","\r\n\r\n ""THE CROW: SALVATION"" -- by Chip...","[, ""THE CROW: SALVATION"" -- by Chip Joha..."
3,Die Hard 2 Script,\r\n\r\n \r\n ...,"[, \n DIE..."
4,Game 6 Script,\r\n \r\n \r\n ...,"[ , \n ..."
...,...,...,...
1172,Ninja Assassin Script,\r\n\r\n\r\n\r\n NINJA...,"[, NINJA ASSASSIN, ..."
1173,"Cooler, The Script",\r\n\r\n\r\n ...,"[, THE..."
1174,Hitchcock Script,\r\n\r\n\r\n\r\n ...,"[, HITCHCOCK..."
1175,Breaking Away Script,"\r\n\r\n \r\n ""BAMB...","[, \n ""BAMBINO"", ..."


In [67]:
#print(dataset_scripts.iloc[0]['script_text'][-1])

TODO
GOAL extract the following features from texts:

Feature engineering TODO list: 
- location: Named Entity Recognition
- character: Named Entity Recognition
    see https://keras.io/examples/nlp/ner_transformers/
    see https://towardsdatascience.com/named-entity-recognition-ner-using-keras-bidirectional-lstm-28cd3f301f54
- convo: ??

sentence embedding?

## Feature Engineering

In [68]:
def str_normalize_eol(s, eol=os.linesep):
    return str.join(eol, str.splitlines(s))

In [69]:
%package joblib

import os
import re

import joblib

class ParagraphExtractor(skl.base.TransformerMixin):
    def __init__(self,
        eol=os.linesep, #'\r\n' #os.linesep
        eol_freq_min=2,
        n_jobs=1
    ):
        self.eol = eol
        self.eol_freq_min = eol_freq_min
        self.n_jobs = n_jobs

        self._pattern = re.compile(
            fr'(?:{self.eol}\s*?){{{self.eol_freq_min},}}'
        )

    def fit(self, _X, _y=None):
        return self

    def transform(self, X, _y=None):
        def _impl_single(s):
            nonlocal self
            return re.split(
                self._pattern,
                str_normalize_eol(s, eol=self.eol)
            )

        return joblib.Parallel(n_jobs=self.n_jobs)(
            joblib.delayed(_impl_single)(s)
                for s in X
        )

        
df_scripts['script_paragraphs'] = ParagraphExtractor(n_jobs=-1).fit_transform(
    df_scripts['script_text']#.iloc[:100]
)

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [70]:
df_scripts_para = (
    df_scripts[['script_name', 'script_paragraphs']]
        .explode('script_paragraphs', ignore_index=True)
)

In [71]:
df_scripts_para['script_paragraphs'] = (
    df_scripts_para['script_paragraphs'].str.strip()
)

In [72]:
df_scripts_para = df_scripts_para[
    # non-empty strings
    df_scripts_para['script_paragraphs'].astype(bool)
]

In [73]:
df_scripts_para

,script_name,script_paragraphs
1,American Milkshake Script,AMERICAN MILKSHAKE
2,American Milkshake Script,Written by
3,American Milkshake Script,David Andalman
4,American Milkshake Script,"Co-writer, Mariko Munro"
6,American Milkshake Script,9/28/2011
...,...,...
1999877,"Sandlot Kids, The Script",makeshit baseball diamond - a sandlot... circa...
1999878,"Sandlot Kids, The Script",A baseball. A baseball with a familiar smudge.
1999879,"Sandlot Kids, The Script",END TITLES.
1999880,"Sandlot Kids, The Script",FADE OUT.


In [74]:
# TODO rm
#df_scripts['script_paragraphs'].iloc[0][:20]

## Models

### Neural Network (LSTM RNN)

#### Text Transformers

In [22]:
class KerasTextVectorizer(skl.base.TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.base = keras.layers.TextVectorization(
            *args, **kwargs
        )

    def fit(self, X, _y=None):
        self.base.adapt(X)
        return self

    def transform(self, X, _y=None):
        return self.base(X)

In [23]:
class SpacyTextVectorizer(KerasTextVectorizer):
    def __init__(self, *args, **kwargs):
        nlp = spacy.load('en_core_web_lg')
        super().__init__(
            *args, 
            vocabulary=list(nlp.vocab.strings),
            **kwargs
        )

    def fit(self, _X, _y=None):
        return self

#### Classifier

In [24]:
def KerasTextClassifier(*args, **kwargs):
    class _class(skeras.wrappers.KerasClassifier):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)

            self.n_features_in_ = None
            self._user_target_encoder = (
                skl.preprocessing.OneHotEncoder(
                    sparse=False
                )
            )

        @property
        def target_encoder(self):
            return self._user_target_encoder

    _inst = None

    def _build_model(optimizer='adam'):
        nonlocal _inst

        n_features = _inst.n_features_in_
        n_classes = np.size(_inst._user_target_encoder.categories_)

        input_dim = n_features
        output_dim = n_classes

        # https://www.tensorflow.org/tutorials/keras/text_classification
        model = keras.Sequential([
            keras.layers.Embedding(
                input_dim=input_dim,
                output_dim=128,
                mask_zero=True
            ),

            keras.layers.Conv1D(
                filters=32, kernel_size=3, 
                padding='same', 
                activation='relu'
            ),
            keras.layers.MaxPooling1D(pool_size=10),
            keras.layers.Dropout(.1),

            keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(64)),

            keras.layers.Dense(128, activation='relu'),

            keras.layers.Dense(
                output_dim,
                activation='softmax'
            )
        ])

        model.compile(
            loss='categorical_crossentropy', 
            optimizer=optimizer, 
            metrics=['accuracy']
        )

        return model

    _inst = _class(
        _build_model, 
        *args, **kwargs
    )
    return _inst

#### Predictor (Vocab From Sample)

In [25]:
def _model_nn(X, y):
    encoder = KerasTextVectorizer(
        max_tokens=None,
        standardize='lower_and_strip_punctuation',
        split='whitespace',
        #ngrams=(1, 2),
        output_mode='int', # TODO tf_idf
        #sparse=True,
    )

    X = encoder.fit_transform(X)

    clf = KerasTextClassifier(
        callbacks=[
            #keras.callbacks.EarlyStopping(
            #    monitor='loss', 
            #    patience=3, 
            #    min_delta=0.0001
            #),
            #keras.callbacks.LambdaCallback(
            #    on_train_end=lambda logs: print(logs)
            #)
        ]
    )
    clf.initialize(X, y)
    
    model = skl.model_selection.GridSearchCV(
        clf,
        param_grid={
            'epochs': [5],
            'batch_size': [1024]
        },
        cv=skl.model_selection.RepeatedStratifiedKFold(
            n_splits=3, n_repeats=1
        ),
        #cv=skl.model_selection.KFold(
        #    n_splits=5, shuffle=True
        #),
        #n_jobs=-1,
        verbose=3
    )

    model.fit(X, y)

    return model

In [30]:
# TODO
df_scripts_para_ = df_scripts_para.sample(5000)
model_nn = _model_nn(
    X=df_scripts_para_['script_paragraphs'],
    y=df_scripts_para_[['script_name']]
)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Epoch 1/5
[CV 1/3] END ...........batch_size=1024, epochs=5;, score=nan total time=   1.5s
Epoch 1/5
[CV 2/3] END ...........batch_size=1024, epochs=5;, score=nan total time=   2.8s
Epoch 1/5
[CV 3/3] END ...........batch_size=1024, epochs=5;, score=nan total time=   1.5s


ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.9/site-packages/scikeras/wrappers.py", line 1494, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/opt/conda/lib/python3.9/site-packages/scikeras/wrappers.py", line 762, in fit
    self._fit(
  File "/opt/conda/lib/python3.9/site-packages/scikeras/wrappers.py", line 931, in _fit
    self._fit_keras_model(
  File "/opt/conda/lib/python3.9/site-packages/scikeras/wrappers.py", line 526, in _fit_keras_model
    hist = se

In [55]:
# TODO histogram
df_scripts['script_paragraphs'].apply(len) < 10

0       False
1       False
2       False
3       False
4       False
        ...  
1203    False
1204    False
1205    False
1206    False
1208    False
Name: script_paragraphs, Length: 1177, dtype: bool

In [82]:
df_scripts[df_scripts['script_paragraphs'].apply(len) < 2]

,script_name,script_text,script_paragraphs
67,Marley & Me Script,,[]
75,Superfights Script,,[]
81,Angels & Demons Script,,[]
91,Kate & Leopold Script,,[]
107,Benny & Joon Script,,[]
124,Equilibrium Script,,[]
156,"Bodies, Rest & Motion Script",,[]
166,Napoleon Dynamite Script,,[]
167,Sneakers Script,,[]
182,Papadopoulos & Sons Script,,[]


In [85]:
#df_scripts.iloc[558]['script_paragraphs']

In [46]:
df_scripts['script_paragraphs'].apply(len).iloc[1175]

1975

In [45]:
len(df_scripts['script_paragraphs'].iloc[1175])

1975